In [2]:
import sys

if ".." not in sys.path:
    sys.path.append("..")

import os
import PIL.Image as Image
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import pickle

from collections import defaultdict

from common.task import path_join, load_ground_truths, get_best_bag_of_features_histograms

%load_ext autoreload
%autoreload 2
%matplotlib widget
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
img_path = '2700270'
n_centroids = 100
step_size = 20
# load ground truth for image 2700270.png
visual_words = load_ground_truths('2700270.gtp') 
visual_word = visual_words[3]
print(f"{visual_word=}")

visual_word=(800, 166, 1007, 246, 'and')


In [4]:
rv = get_best_bag_of_features_histograms('2700270.png', visual_word[:4], n_centroids, step_size)

Using cell_size=80 and step_size=20
Computing sift descriptors and pickling afterwards...


FileNotFoundError: [Errno 2] No such file or directory: '/Users/dh/projects/group_fpda/data/gw/pickle_data/sift_descriptors/2700270_80_sift-20_descriptors.p'

# Visualization

In [ ]:
x1, y1, x2, y2, _ = visual_word
document = Image.open(path_join('pages', '2700270.png'))
doc_arr = np.asarray(document, dtype='uint8')
req_arr = doc_arr[y1:y2, x1:x2]

plt.imshow(req_arr, cmap=cm.get_cmap("Greys_r"))
plt.title("Request image")
plt.show()

In [ ]:
row_cols = 4
fig = plt.figure(figsize=(10, 6))
fig.suptitle("Best matching windows in document")

for idx, info in enumerate(rv):
    if idx == row_cols**2:
        break

    wx, wy, wxx, wyy = info['window']
    ax = fig.add_subplot(row_cols, row_cols, idx + 1)
    subimg = doc_arr[wy:wyy, wx:wxx]
    ax.imshow(subimg, cmap=cm.get_cmap('Greys_r'))

In [ ]:
total_word_occurences = list(filter(lambda x: x[-1] == visual_word[-1], visual_words))
print(f"{len(total_word_occurences)=}")
total_word_occurences

# Evaluation

In [ ]:
from common.evaluation import SegmentfreeWordSpottingEvaluator

img_path = '2700270.png'
n_centroids = 100
step_size = 25
word_index = 3
max_eval_length = 16

sfwse = SegmentfreeWordSpottingEvaluator(
    img_path,
    n_centroids,
    step_size
)

result = sfwse.crossvalidate(word_index, max_eval_length)

In [ ]:
prec, rec, mean_prec = result
print(f"{prec=} {rec=} {mean_prec=}")

In [ ]:
from common.evaluation import SegmentfreeWordSpottingEvaluator

# TODO: run on server
img_path = '2700270.png'
n_centroids = 100
step_size = 15
word_index = 8
max_eval_length = 10

sfwse = SegmentfreeWordSpottingEvaluator(
    img_path,
    n_centroids,
    step_size,
    plot_results=False
)

srv = sfwse.validate(max_eval_length)

subpath = f'srv_{img_path}_{n_centroids}_{step_size}_{word_index}_{max_eval_length}.p'
with open(path_join('pickle_data', subpath), 'wb') as fh:
    pickle.dump(srv, fh)

In [ ]:
from common.evaluation import SegmentfreeWordSpottingEvaluator

img_path = '2700270.png'
n_centroids = 100
step_size = 50
word_index = 8
max_eval_length = 10

sfwse = SegmentfreeWordSpottingEvaluator(
    img_path,
    n_centroids,
    step_size,
    plot_results=False
)

rv = sfwse.validate(max_eval_length)
subpath = f'rv_{img_path}_{n_centroids}_{step_size}_{word_index}_{max_eval_length}.p'
with open(path_join('pickle_data', subpath), 'wb') as fh:
    pickle.dump(rv, fh)

In [ ]:
from distributed import Client